## Run training scripts for Models to measure performance
---------------------

This notebook initiates the training script to execute and measures metrics such as time to train, loss values, instance utilization metrics, etc. 

The training pipeline consists of the following stages:

1. Configuration Retrieval: Experiment configurations, including training scripts, hyperparameters, and datasets, are fetched from a central repository or file.

1. Training Execution: The training script is executed based on the configuration. This script:

1. Prepares the dataset.

1. Sets up the model and tokenizer.

1. Initiates training using the optimum.neuron.NeuronTrainer.

1. Saves the trained model and logs metrics.

1. Metrics Recording: Metrics such as loss, training time, and validation accuracy are captured and stored.

All the data and metrics are stored in the `report.html` that is created. User can configure variables for target total loss value as well as the time to train in the config file.

#### Import all of the necessary libraries below to run this notebook

In [1]:
# if interactive mode is set to no -> pickup fmbench from Python installation path
# if interactive mode is set to yes -> pickup fmbench from the current path (one level above this notebook)
# if interactive mode is not defined -> pickup fmbench from the current path (one level above this notebook)
# the premise is that if run non-interactively then it can only be run through main.py which will set interactive mode to no
import os
import sys
if os.environ.get("INTERACTIVE_MODE_SET", "yes") == "yes":
    sys.path.append(os.path.dirname(os.getcwd()))

In [2]:
import sys
import time
import json
import boto3
import asyncio
import logging
import importlib.util
import fmbench.scripts
from pathlib import Path
from fmbench.utils import *
from fmbench.globals import *
from fmbench.scripts import constants
from typing import Dict, List, Optional
from sagemaker import get_execution_role
import importlib.resources as pkg_resources
from botocore.exceptions import ClientError
from botocore.exceptions import NoCredentialsError

region_name=us-east-1
role_arn_from_env=None, using current sts caller identity to set arn_string
the sts role is an assumed role, setting arn_string to arn:aws:iam::218208277580:role/fmbench-new-us-east-1-role
config file current -> configs/training/config-train-llama3-8b-optimum-neuron.yml, None
loaded config: {'general': {'name': 'llama3-8b-g6e.2xl-ec2', 'model_name': 'llama3-8b-instruct'}, 'aws': {'region': 'us-east-1', 'sagemaker_execution_role': 'arn:aws:iam::218208277580:role/fmbench-new-us-east-1-role', 'bucket': 'sagemaker-fmbench-write-us-east-1-218208277580'}, 'dir_paths': {'data_prefix': 'data', 'prompts_prefix': 'prompts', 'all_prompts_file': 'all_prompts.csv', 'metrics_dir': 'metrics', 'models_dir': 'models', 'metadata_dir': 'metadata'}, 's3_read_data': {'read_bucket': 'sagemaker-fmbench-read-us-east-1-218208277580', 'scripts_prefix': 'scripts', 'script_files': ['hf_token.txt'], 'configs_prefix': 'configs', 'config_files': ['pricing.yml'], 'source_data_prefix': 'source_da

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


/home/ec2-user/anaconda3/envs/fmbench_python311/lib/python3.11/site-packages/fmbench/scripts/hf_token.txt file not found
CustomTokenizer, based on HF transformers, sagemaker-fmbench-read-us-east-1-218208277580 prefix: llama3_tokenizer local_dir: tokenizer, model_id: meta-llama/Meta-Llama-3-8B-Instruct
CustomTokenizer, all_files = [PosixPath('tokenizer/config.json'), PosixPath('tokenizer/tokenizer.json')]
loading the provided tokenizer from local_dir=tokenizer, abs_path=/home/ec2-user/SageMaker/training_benchmarking/foundation-model-benchmarking-tool/src/fmbench/tokenizer
successfully loaded the tokenizer using AutoTokenizer.from_pretrained from tokenizer
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


#### Set up a logger to log all messages while the code runs

In [3]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

### Load the config.yml file
------

The config.yml file contains information that is used across this benchmarking environment, such as information about the aws account, prompts, payloads to be used for invocations, and model configurations like the version of the model, the endpoint name, model_id that needs to be deployed. Configurations also support the gives instance type to be used, for example: "ml.g5.24xlarge", the image uri, whether or not to deploy this given model, followed by an inference script "jumpstart.py" which supports the inference script for jumpstart models to deploy the model in this deploy notebook. The experiment configuration also has the training script parameter, that is run as a part of the benchmarking test, where the loss value, time to train and other metrics are recorded.

In [4]:
## Load the config.yml file referring to the globals.py file
config = load_main_config(CONFIG_FILE)

## configure the aws region and execution role
aws_region = config['aws']['region']


# try:
#     sagemaker_execution_role = get_execution_role()
#     config['aws']['sagemaker_execution_role'] = sagemaker_execution_role
#     logger.info(f"determined SageMaker exeuction role from get_execution_role")
# except Exception as e:
#     logger.error(f"could not determine SageMaker execution role, error={e}")
#     logger.info(f"going to look for execution role in config file..")
#     sagemaker_execution_role = config['aws'].get('sagemaker_execution_role')
#     if sagemaker_execution_role is not None:
#         logger.info(f"found SageMaker execution role in config file..")

logger.info(f"aws_region={aws_region}, execution_role={config['aws']['sagemaker_execution_role']}")
logger.info(f"config={json.dumps(config, indent=2)}")

[2025-01-18 00:12:30,622] p16712 {1346162553.py:19} INFO - aws_region=us-east-1, execution_role=arn:aws:iam::218208277580:role/fmbench-new-us-east-1-role
[2025-01-18 00:12:30,623] p16712 {1346162553.py:20} INFO - config={
  "general": {
    "name": "llama3-8b-g6e.2xl-ec2",
    "model_name": "llama3-8b-instruct"
  },
  "aws": {
    "region": "us-east-1",
    "sagemaker_execution_role": "arn:aws:iam::218208277580:role/fmbench-new-us-east-1-role",
    "bucket": "sagemaker-fmbench-write-us-east-1-218208277580"
  },
  "dir_paths": {
    "data_prefix": "data",
    "prompts_prefix": "prompts",
    "all_prompts_file": "all_prompts.csv",
    "metrics_dir": "metrics",
    "models_dir": "models",
    "metadata_dir": "metadata"
  },
  "s3_read_data": {
    "read_bucket": "sagemaker-fmbench-read-us-east-1-218208277580",
    "scripts_prefix": "scripts",
    "script_files": [
      "hf_token.txt"
    ],
    "configs_prefix": "configs",
    "config_files": [
      "pricing.yml"
    ],
    "source_data

region_name=us-east-1
role_arn_from_env=None, using current sts caller identity to set arn_string
the sts role is an assumed role, setting arn_string to arn:aws:iam::218208277580:role/fmbench-new-us-east-1-role


In [5]:
def run_training(experiment_config: Dict, aws_region: str, role_arn: str) -> Optional[Dict]:
    """Function to run training for a model"""
    # Log the training details
    logger.info(f"going to train {experiment_config}, in {aws_region} with {role_arn}")
    training_result: Optional[Dict] = None

    # Check if training is enabled in the config; skip if not
    run_training = experiment_config.get('run_training', False)
    if run_training is False:
        logger.info(f"skipping training of {experiment_config['model_id']} because run_training={run_training}")
        training_result = dict(
            model_id=experiment_config['model_id'],
            experiment_name=experiment_config['name'],
            instance_type=experiment_config.get('instance_type'),
            instance_count=experiment_config.get('instance_count'),
            trained=False
        )
        return training_result

    # Get the scripts directory
    scripts_path = Path(pkg_resources.files('fmbench'), 'scripts')
    training_dir = scripts_path / 'training_scripts'
    logger.info(f"Using fmbench.scripts directory: {training_dir}")

    try:
        # Import and run the training script
        module_name = Path(experiment_config['training_script']).stem
        logger.info(f"script provided for training this model is --> {module_name}")
        training_script_path = training_dir / f"{module_name}.py"
        logger.info(f"script path is --> {training_script_path}")

        if not training_script_path.exists():
            logger.error(f"Training script {training_script_path} not found.")
            return None
        logger.info(f"Training using local code: {training_script_path}")

        # Import the training module
        spec = importlib.util.spec_from_file_location(module_name, str(training_script_path))
        module = importlib.util.module_from_spec(spec)
        sys.modules[module_name] = module
        spec.loader.exec_module(module)

        # Run training and measure time
        st = time.perf_counter()
        training_result = module.train(experiment_config, role_arn)
        elapsed_time = time.perf_counter() - st
        logger.info(f"time taken to train model_id={experiment_config['model_id']} via "
                    f"{training_script_path} is {elapsed_time:0.2f}")
        # Add training time to result
        if training_result:
            training_result['training_time'] = elapsed_time
            training_result['trained'] = True
        return training_result

    except Exception as error:
        logger.error(f"An error occurred during training: {error}")
        return training_result

In [6]:
async def async_run_training(experiment_config: Dict, role_arn: str, aws_region: str) -> Dict:
    """Asynchronous wrapper function to allow concurrent training requests"""
    return await asyncio.to_thread(run_training, experiment_config, role_arn, aws_region)

async def async_train_all_models(config: Dict) -> List[Dict]:
    """Final asynchronous function to train all models concurrently"""
    # Extract experiments from the config
    experiments: List[Dict] = config['experiments']
    n: int = 4  # max concurrency to avoid throttling
    # Check for non-reentrant training scripts
    non_reentrant_training_scripts = config.get('non_reentrant_training_scripts', [])
    non_reentrant_scripts_present = [e['training_script'] for e in experiments 
                                   if e['training_script'] in non_reentrant_training_scripts]
    if len(non_reentrant_scripts_present) > 1:
        logger.info(f"non_reentrant_training_scripts_present={len(non_reentrant_scripts_present)}, "
                    f"going to train models serially")
        n = 1

    # Split experiments into smaller batches for concurrent training
    experiments_splitted = [experiments[i * n:(i + 1) * n] for i in range((len(experiments) + n - 1) // n)]
    results = []
    for exp_list in experiments_splitted:
        # Run training in batches
        result = await asyncio.gather(*[
            async_run_training(
                m,
                config['aws']['region'],
                config['aws']['sagemaker_execution_role']
            ) for m in exp_list
        ])
        # Collect results from each batch
        results.extend(result)
    return results

In [7]:
# Start timer
s = time.perf_counter()

# Run all training jobs
training_results = await async_train_all_models(config)

# Calculate total time
elapsed_async = time.perf_counter() - s
logger.info(f"Training results: {training_results}")
logger.info(f"All training completed in {elapsed_async:0.2f} seconds")

[2025-01-18 00:12:33,256] p16712 {1365297455.py:4} INFO - going to train {'name': 'llama3-8b-instruct', 'region': 'us-east-1', 'model_id': 'meta-llama/Meta-Llama-3-8B-Instruct', 'model_version': None, 'model_name': 'llama3-8b-instruct', 'ep_name': 'http://127.0.0.1:8080/invocations', 'instance_type': 'g6e.2xlarge', 'image_uri': '763104351884.dkr.ecr.us-east-1.amazonaws.com/djl-inference:0.29.0-lmi11.0.0-cu124', 'deploy': True, 'instance_count': None, 'deployment_script': 'ec2_deploy.py', 'inference_script': 'ec2_predictor.py', 'training_script': 'finetune_hf_optimum_neuron.py', 'run_training': True, 'training_hyperparameters': None, 'ec2': {'gpu_or_neuron_setting': '--gpus all --shm-size 12g', 'model_loading_timeout': 2400}, 'inference_spec': {'parameter_set': 'ec2_djl', 'tp_degree': 1, 'shm_size': '12g', 'model_loading_timeout': 2400, 'parameters': {'do_sample': True, 'temperature': 0.1, 'top_p': 0.92, 'top_k': 120, 'max_new_tokens': 100, 'Content-type': 'application/json', 'return_fu